# Final Training

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import torch
import albumentations as A

from scripts.models import CoinClassifier
from scripts.training import train_model, save_trainable_params
from scripts.utils import split_data, get_images_from_coco, ClassificationDataset, setup_seed

from torch.utils.data import DataLoader
from torch import nn
from pathlib import Path
from torch.optim.lr_scheduler import CosineAnnealingLR

OSError: dlopen(/Users/jan.kokla/miniconda3/envs/iapr_project/lib/python3.9/site-packages/torchaudio/_torchaudio.so, 0x0006): Symbol not found: __ZNK3c104Type14isSubtypeOfExtERKNSt3__110shared_ptrIS0_EEPNS1_13basic_ostreamIcNS1_11char_traitsIcEEEE
  Referenced from: <D1FEA5E2-D31D-3375-AD47-CD1FABF9F7C5> /Users/jan.kokla/miniconda3/envs/iapr_project/lib/python3.9/site-packages/torchaudio/_torchaudio.so
  Expected in:     <12FB406D-F8AC-3782-A1E8-ADE5A43768BA> /Users/jan.kokla/miniconda3/envs/iapr_project/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib

## Hyperparameters and Config

In [6]:
setup_seed(13)

In [20]:
ROOT = Path(os.path.abspath('')).parent

# let's specify paths to training images and masks
images_path = "../data/train"
coins_path = "../data/classification"
labels_path = "../data/classification/labels.json"

annotation_path = "../data/annotations.json"
cls_path = "../data/classification"

In [8]:
# training hyperparameters
batch_size = 8

## Transformations

We first define data augmentations for more robust model. Since our backbones are pretrained on Imagenet, we apply dataset specific normalization to the transformations.

In [9]:
# cut coins from images if necessary and save them as separate images
get_images_from_coco(images_path, annotation_path, cls_path)

Files already there, good to go!


In [24]:
# imagenet specific normalization (was applied also during backbone training)
normalization_kwargs = {
    'mean': (0.485, 0.456, 0.406), 
    'std': (0.229, 0.224, 0.225), 
    'always_apply': True
}

# define data augmentation for train and validation
train_tf = A.Compose([
    A.Resize(width=224, height=224, always_apply=True),
    A.RandomRotate90(p=0.9),
    A.RandomBrightnessContrast(p=0.5),
    A.Blur(blur_limit=3),
    A.OpticalDistortion(),
    A.Normalize(**normalization_kwargs)
])

## Modelling

### Predicting Currency

We first build a classifier that will separate between EUR and CHF.

#### Datasets and DataLoaders

In [25]:
# split the image paths into train and validation
images_train, images_val, labels_train, labels_val = split_data(
    coins_path, 0, "classification", labels_path, "ccy"
)

# get train and val dataset instances
train_ds = ClassificationDataset(
    image_paths=images_train,
    labels=labels_train,
    transform=train_tf,
)

train_loader = DataLoader(train_ds, batch_size=batch_size)

#### Modelling

We use `Segmentation Models Pytorch` for model generation, then we define training arguments and train the model

In [26]:
model = CoinClassifier(num_classes=2, coin_type="ccy")

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.model.head.parameters(), lr=0.0005)
scheduler = CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-6)

In [28]:
# train the model
_ = train_model(
    model,
    (train_loader, None),
    criterion,
    optimizer,
    scheduler,
    num_epochs=30
)

Epoch:   1. Train.      Loss: 0.554 | f1: 0.704: 100%|██████████| 42/42 [00:05<00:00,  7.37it/s]
Epoch:   2. Train.      Loss: 0.268 | f1: 0.964: 100%|██████████| 42/42 [00:05<00:00,  7.45it/s]
Epoch:   3. Train.      Loss: 0.174 | f1: 0.988: 100%|██████████| 42/42 [00:05<00:00,  7.67it/s]
Epoch:   4. Train.      Loss: 0.137 | f1: 0.970: 100%|██████████| 42/42 [00:05<00:00,  7.68it/s]
Epoch:   5. Train.      Loss: 0.104 | f1: 0.988: 100%|██████████| 42/42 [00:05<00:00,  7.37it/s]
Epoch:   6. Train.      Loss: 0.090 | f1: 0.991: 100%|██████████| 42/42 [00:05<00:00,  7.39it/s]
Epoch:   7. Train.      Loss: 0.085 | f1: 0.991: 100%|██████████| 42/42 [00:05<00:00,  7.22it/s]
Epoch:   8. Train.      Loss: 0.071 | f1: 0.991: 100%|██████████| 42/42 [00:06<00:00,  6.98it/s]
Epoch:   9. Train.      Loss: 0.066 | f1: 0.994: 100%|██████████| 42/42 [00:06<00:00,  6.43it/s]
Epoch:  10. Train.      Loss: 0.062 | f1: 0.991: 100%|██████████| 42/42 [00:06<00:00,  6.49it/s]
Epoch:  11. Train.      Loss: 

In [29]:
filepath = f'{ROOT}/models/{model.task}_{model.coin_type}.pt'
save_trainable_params(model, filepath)

### Predicting EUR

Now we can build a classifier to distinguish between EUR coin types.

#### Datasets and DataLoaders

In [1]:
# split the image paths into train and validation
images_train, _, labels_train, _ = split_data(
    coins_path, 0, "classification", labels_path, "eur"
)

# get train and val dataset instances
train_ds = ClassificationDataset(
    image_paths=images_train,
    labels=labels_train,
    transform=train_tf
)

# create dataloaders
train_loader = DataLoader(train_ds, batch_size)

NameError: name 'split_data' is not defined

#### Modelling

In [ ]:
model = CoinClassifier(num_classes=8, coin_type="eur", freeze=False)

In [2]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00003)
scheduler = CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-6)

NameError: name 'nn' is not defined

In [36]:
_ = train_model(
    model,
    (train_loader, None),
    criterion,
    optimizer,
    scheduler,
    num_epochs=26
)

Epoch:   1. Train.      Loss: 1.934 | f1: 0.280: 100%|██████████| 25/25 [00:12<00:00,  2.08it/s]
Epoch:   2. Train.      Loss: 1.175 | f1: 0.550: 100%|██████████| 25/25 [00:11<00:00,  2.12it/s]
Epoch:   3. Train.      Loss: 0.675 | f1: 0.825: 100%|██████████| 25/25 [00:12<00:00,  2.00it/s]
Epoch:   4. Train.      Loss: 0.458 | f1: 0.910: 100%|██████████| 25/25 [00:13<00:00,  1.84it/s]
Epoch:   5. Train.      Loss: 0.279 | f1: 0.960: 100%|██████████| 25/25 [00:14<00:00,  1.71it/s]
Epoch:   6. Train.      Loss: 0.181 | f1: 0.980: 100%|██████████| 25/25 [00:15<00:00,  1.66it/s]
Epoch:   7. Train.      Loss: 0.133 | f1: 0.980: 100%|██████████| 25/25 [00:15<00:00,  1.63it/s]
Epoch:   8. Train.      Loss: 0.111 | f1: 0.980: 100%|██████████| 25/25 [00:14<00:00,  1.70it/s]
Epoch:   9. Train.      Loss: 0.077 | f1: 0.990: 100%|██████████| 25/25 [00:14<00:00,  1.76it/s]
Epoch:  10. Train.      Loss: 0.082 | f1: 0.980: 100%|██████████| 25/25 [00:14<00:00,  1.76it/s]
Epoch:  11. Train.      Loss: 

In [37]:
filepath = f'{ROOT}/models/{model.task}_{model.coin_type}.pt'
save_trainable_params(model, filepath)

### Predicting CHF heads or tails

#### Datasets and DataLoaders

In [38]:
# split the image paths into train and validation
labels_path = "../data/classification/heads_tails.json"
images_train, _, labels_train, _ = split_data(
    coins_path, 0, "classification", labels_path, "heads-tails"
)

# get train and val dataset instances
train_ds = ClassificationDataset(
    image_paths=images_train,
    labels=labels_train,
    transform=train_tf,
)

# create dataloaders
train_loader = DataLoader(train_ds, batch_size)

#### Modelling

In [39]:
model = CoinClassifier(num_classes=2, coin_type="heads-tails", freeze=False)

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.model.parameters(), lr=0.00005)
scheduler = CosineAnnealingLR(optimizer, T_max=2)

In [41]:
# train the model
_ = train_model(
    model,
    (train_loader, None),
    criterion,
    optimizer,
    scheduler,
    num_epochs=10
)

Epoch:   1. Train.      Loss: 0.556 | f1: 0.763: 100%|██████████| 17/17 [00:07<00:00,  2.19it/s]
Epoch:   2. Train.      Loss: 0.110 | f1: 0.978: 100%|██████████| 17/17 [00:05<00:00,  2.84it/s]
Epoch:   3. Train.      Loss: 0.041 | f1: 0.985: 100%|██████████| 17/17 [00:06<00:00,  2.81it/s]
Epoch:   4. Train.      Loss: 0.025 | f1: 0.993: 100%|██████████| 17/17 [00:06<00:00,  2.79it/s]
Epoch:   5. Train.      Loss: 0.010 | f1: 1.000: 100%|██████████| 17/17 [00:05<00:00,  2.97it/s]
Epoch:   6. Train.      Loss: 0.007 | f1: 1.000: 100%|██████████| 17/17 [00:06<00:00,  2.49it/s]
Epoch:   7. Train.      Loss: 0.030 | f1: 0.985: 100%|██████████| 17/17 [00:07<00:00,  2.28it/s]
Epoch:   8. Train.      Loss: 0.052 | f1: 0.993: 100%|██████████| 17/17 [00:08<00:00,  2.08it/s]
Epoch:   9. Train.      Loss: 0.025 | f1: 0.993: 100%|██████████| 17/17 [00:08<00:00,  2.10it/s]
Epoch:  10. Train.      Loss: 0.034 | f1: 0.993: 100%|██████████| 17/17 [00:08<00:00,  2.08it/s]


In [42]:
filepath = f'{ROOT}/models/{model.task}_{model.coin_type}.pt'
save_trainable_params(model, filepath)

### Predicting CHF tails

#### Datasets and DataLoaders

In [43]:
# split the image paths into train and validation
labels_path = "../data/classification/chf_tails.json"
images_train, _, labels_train, _ = split_data(
    coins_path, 0, "classification", labels_path, "chf-tails"
)

# get train and val dataset instances
train_ds = ClassificationDataset(
    image_paths=images_train,
    labels=labels_train,
    transform=train_tf,
)

# create dataloaders
train_loader = DataLoader(train_ds, batch_size=8)

#### Modelling

In [53]:
model = CoinClassifier(num_classes=7, coin_type="chf-tails", freeze=False)

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.model.parameters(), lr=0.00005, weight_decay=0.2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=3)

In [55]:
_ = train_model(
    model,
    (train_loader, None),
    criterion,
    optimizer,
    scheduler,
    num_epochs=26
)

Epoch:   1. Train.      Loss: 1.398 | f1: 0.450: 100%|██████████| 5/5 [00:04<00:00,  1.15it/s]
Epoch:   2. Train.      Loss: 0.814 | f1: 0.675: 100%|██████████| 5/5 [00:02<00:00,  2.18it/s]
Epoch:   3. Train.      Loss: 0.584 | f1: 0.850: 100%|██████████| 5/5 [00:02<00:00,  2.50it/s]
Epoch:   4. Train.      Loss: 0.427 | f1: 0.900: 100%|██████████| 5/5 [00:01<00:00,  2.51it/s]
Epoch:   5. Train.      Loss: 0.346 | f1: 0.900: 100%|██████████| 5/5 [00:02<00:00,  2.43it/s]
Epoch:   6. Train.      Loss: 0.220 | f1: 0.975: 100%|██████████| 5/5 [00:02<00:00,  2.39it/s]
Epoch:   7. Train.      Loss: 0.229 | f1: 0.975: 100%|██████████| 5/5 [00:02<00:00,  2.33it/s]
Epoch:   8. Train.      Loss: 0.181 | f1: 1.000: 100%|██████████| 5/5 [00:02<00:00,  2.27it/s]
Epoch:   9. Train.      Loss: 0.158 | f1: 1.000: 100%|██████████| 5/5 [00:02<00:00,  2.25it/s]
Epoch:  10. Train.      Loss: 0.119 | f1: 1.000: 100%|██████████| 5/5 [00:02<00:00,  2.28it/s]
Epoch:  11. Train.      Loss: 0.103 | f1: 1.000: 1

In [56]:
filepath = f'{ROOT}/models/{model.task}_{model.coin_type}.pt'
save_trainable_params(model, filepath)

### Predicting CHF heads

#### Datasets and Dataloaders

In [57]:
# split the image paths into train and validation
labels_path = "../data/classification/chf_heads.json"
images_train, _, labels_train, _ = split_data(
    coins_path, 0, "classification", labels_path, "chf-heads"
)

# get train and val dataset instances
train_ds = ClassificationDataset(
    image_paths=images_train,
    labels=labels_train,
    transform=train_tf,
)

# create dataloaders
train_loader = DataLoader(train_ds, batch_size)

#### Modelling

In [58]:
model = CoinClassifier(num_classes=3, coin_type="chf-heads", freeze=False)

In [59]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.model.parameters(), lr=0.00001)
scheduler = CosineAnnealingLR(optimizer, T_max=3, eta_min=1e-6)

In [60]:
_ = train_model(
    model,
    (train_loader, None),
    criterion,
    optimizer,
    scheduler,
    num_epochs=30
)

Epoch:   1. Train.      Loss: 1.025 | f1: 0.535: 100%|██████████| 6/6 [00:02<00:00,  2.05it/s]
Epoch:   2. Train.      Loss: 0.804 | f1: 0.674: 100%|██████████| 6/6 [00:02<00:00,  2.98it/s]
Epoch:   3. Train.      Loss: 0.694 | f1: 0.757: 100%|██████████| 6/6 [00:02<00:00,  2.99it/s]
Epoch:   4. Train.      Loss: 0.547 | f1: 0.854: 100%|██████████| 6/6 [00:02<00:00,  2.90it/s]
Epoch:   5. Train.      Loss: 0.502 | f1: 0.917: 100%|██████████| 6/6 [00:02<00:00,  2.96it/s]
Epoch:   6. Train.      Loss: 0.421 | f1: 0.917: 100%|██████████| 6/6 [00:02<00:00,  2.86it/s]
Epoch:   7. Train.      Loss: 0.351 | f1: 0.938: 100%|██████████| 6/6 [00:02<00:00,  2.84it/s]
Epoch:   8. Train.      Loss: 0.334 | f1: 0.938: 100%|██████████| 6/6 [00:02<00:00,  3.00it/s]
Epoch:   9. Train.      Loss: 0.273 | f1: 0.979: 100%|██████████| 6/6 [00:02<00:00,  2.97it/s]
Epoch:  10. Train.      Loss: 0.187 | f1: 1.000: 100%|██████████| 6/6 [00:01<00:00,  3.02it/s]
Epoch:  11. Train.      Loss: 0.163 | f1: 1.000: 1

In [61]:
filepath = f'{ROOT}/models/{model.task}_{model.coin_type}.pt'
save_trainable_params(model, filepath)